In [ ]:
import gzip
import numpy as np
from collections import defaultdict
from sklearn.metrics import mean_squared_error

In [ ]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [37]:
from random import shuffle
data = []
for l in readGz("train.json.gz"):
    data.append(l)

print(data[0])
shuffle(data)
print(data[0])

#trainData, validData = np.array_split(data, 2)
trainData, validData = data, data[:10000]

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}
{'rating': 5.0, 'businessID': 'B154066252', 'reviewText': "this is one of oc favorites. try the fried oysters. some of the best raw oysters around. the rice salad is outstanding. the rib eye is very good, and the fish is always good. try the shoe string potatoes. excellent bar, serving good cosmos and mojitos. a very busy restaurant - so be prepared to wait. owned by the group that owns houston's, cafe r&d, and bandera.", 'userID': 'U874711717', 'reviewTime': 'Mar 15, 2006', 'categories': ['Seafood Restaurant', 'American Restaurant'], 'reviewHash': 'R090598088', 'uni

In [38]:
allRatings = []
userRatings = defaultdict(dict)
busRatings = defaultdict(dict)
globalAvg = 0.0
userIndex = defaultdict(int)
busIndex = defaultdict(int)

uCount = 0
bCount = 0

for d in trainData:
    user, bus, rating = d['userID'], d['businessID'], d['rating']
    allRatings.append(rating)
    globalAvg += rating
    userRatings[user][bus] = rating
    busRatings[bus][user] = rating
    if user not in userIndex:
        userIndex[user] = uCount
        uCount += 1
    if bus not in busIndex:
        busIndex[bus] = bCount
        bCount += 1

numUsers = len(userRatings) #storing the number of unique users in training set
numBus = len(busRatings) #storing the number of unique users in training set

print(len(userIndex))
print(numUsers)
print(len(busIndex))
print(numBus)

lenTData = len(allRatings)

globalAvg = globalAvg/lenTData

vallRatings = []
vuserRatings = defaultdict(dict)
vbusRatings = defaultdict(dict)

for d in validData:
    user, bus, rating = d['userID'], d['businessID'], d['rating']
    vallRatings.append(rating)
    vuserRatings[user][bus] = rating
    vbusRatings[bus][user] = rating
    
print(vallRatings[0])

18793
18793
21321
21321
5.0


In [39]:
alpha = 0;
betaU = defaultdict(int)
betaI = defaultdict(int)
lowDimSize = 10
gammaU = defaultdict(lambda: [1.0/lowDimSize for x in range(lowDimSize)])
gammaI = defaultdict(lambda: [1.0/lowDimSize for x in range(lowDimSize)])
lamda = 10

def resetVars(lSize, lamdanew):
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize, globalAvg, lamda
    lowDimSize = lSize
    alpha = 0.0
    lamda = lamdanew
    betaU = defaultdict(int)
    betaI = defaultdict(int)
    gammaU = defaultdict(lambda: np.array([1.0/lowDimSize for x in range(lowDimSize)]))
    gammaI = defaultdict(lambda: np.array([1.0/lowDimSize for x in range(lowDimSize)]))

In [40]:
predRatings = []
trainRatingTuples = [[d['userID'], d['businessID'], d['rating']] for d in trainData]
print(trainRatingTuples[0][0])

def predictValid():
    global predRatings
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize
    predRatings = []
    for d in validData:
        user, business = d['userID'], d['businessID']
        prediction = alpha + betaU[user] + betaI[business] + np.matmul(gammaU[user],np.transpose(gammaI[bus]))
        predRatings.append(prediction[0])
    #print(len(vallRatings))
    #print(len(predRatings))
    #print(predRatings[0])
    MSE_alpha = mean_squared_error(vallRatings, predRatings)
    return MSE_alpha

def trainLatentFactor(updateGammaU):
    global alpha, betaU, betaI, gammaU, gammaI, predRatings, lowDimSize, lamda
    
    #updating alpha
    
    newAlpha = 0.0
    for d in range(lenTData):
        user, business, rating = trainRatingTuples[d][0], trainRatingTuples[d][1], trainRatingTuples[d][2]
        gammaProd = np.matmul(gammaU[user], np.transpose(gammaI[business]))
        newAlpha += (userRatings[user][business] - betaU[user] - betaU[business] - gammaProd)
    newAlpha /= lenTData
    alpha = newAlpha
    
    #updating betaU
    for u in userRatings:
        tempBeta = 0.0
        busCount = 0
        for b in userRatings[u]:
            gammaProd = np.matmul(gammaU[u], np.transpose(gammaI[b]))
            tempBeta += (userRatings[u][b] - alpha - betaI[b] - gammaProd)
            busCount += 1
        tempBeta /= (lamda + busCount)
        betaU[u] = tempBeta
        
    #updating betaI
    for b in busRatings:
        tempBeta = 0.0
        uCount = 0
        for u in busRatings[b]:
            gammaProd = np.matmul(gammaU[u], np.transpose(gammaI[b]))
            tempBeta += (userRatings[u][b] - alpha - betaU[u] - gammaProd)
            uCount += 1
        tempBeta /= (lamda + uCount)
        betaI[b] = tempBeta
        
    if(updateGammaU):
        for u in userRatings:
            newGamma = np.zeros(shape=(1,lowDimSize))
            gammaProd = 0.0
            for b in userRatings[u]:
                temp = (userRatings[u][b] - alpha - betaU[u] - betaI[b])
                temp = temp*gammaI[b]
                newGamma = newGamma + temp
                gammaProd += np.matmul(gammaI[b], np.transpose(gammaI[b]))
            newGamma = newGamma/ (gammaProd + lamda)
            gammaU[u] = newGamma
    else:
        for b in busRatings:
            newGamma = np.zeros(shape=(1, lowDimSize))
            gammaProd = 0.0
            for u in busRatings[b]:
                newGamma = newGamma + (userRatings[u][b] - alpha - betaU[u] - betaI[b])*gammaU[u]
                gammaProd += np.matmul(gammaU[u], np.transpose(gammaU[u]))
            newGamma = newGamma / (gammaProd + lamda)
            gammaI[b] = newGamma
        
def train(i, k, lamdat):
    global lowDimSize, lamda
    lowDimSize = k
    lamda = lamdat
    for x in range(i):
        if(x%2==0):
            trainLatentFactor(True)
        else:
            trainLatentFactor(False)
    print(lamdat)
    print(predictValid())

U874711717


In [14]:
dimSize = 1
lams = 4.0
for l in range(10):
    resetVars(dimSize, lams)
    print("Lamda value: "+str(lams))
    train(10, dimSize, lams)
    lams += 0.1
    print("Alpha value: " +str(alpha))

Lamda value: 4.5
4.5
0.60635651172
Alpha value: [[ 4.25708744]]


In [36]:
#resetVars(2, 4.5)
#train(60, 2, 4.5)

4.5
0.609137951299


In [41]:
resetVars(1, 4.5)
train(100, 1, 4.5)

4.5
0.454873279147


In [ ]:
resetVars(2, 4.5)
train(100, 2, 4.5)

In [42]:
def ratingPredFile(inputFile, outputFile):
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize
    predictions = open(outputFile, 'w')
    for l in open(inputFile):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        user,business = l.strip().split('-')
        prediction = 0.0
        prediction = prediction + alpha
        if user in betaU and business in betaI:
            prediction += (betaU[user] + betaI[business] + np.matmul(gammaU[user], np.transpose(gammaI[business])))
            print(prediction)
        elif user in betaU:
            prediction += betaU[user]
        elif business in betaI:
            prediction += betaI[business]
        
        predictions.write(user + '-' + business + "," + str(prediction[0][0])+"\n")

ratingPredFile("pairs_Rating.txt","predictions_Rating.txt")
print("done")

[[ 2.9974231]]
[[ 4.76069803]]
[[ 4.3867948]]
[[ 3.96895147]]
[[ 3.97003458]]
[[ 4.68581812]]
[[ 3.1006323]]
[[ 4.640512]]
[[ 4.60790141]]
[[ 3.82641964]]
[[ 4.80671447]]
[[ 4.28697824]]
[[ 4.29967506]]
[[ 3.79226296]]
[[ 3.94715462]]
[[ 3.99337832]]
[[ 3.25385961]]
[[ 4.20706917]]
[[ 4.39283491]]
[[ 3.550783]]
[[ 4.23996141]]
[[ 4.5325532]]
[[ 4.00471428]]
[[ 4.68404838]]
[[ 4.01674015]]
[[ 3.96858024]]
[[ 4.30763758]]
[[ 4.43163697]]
[[ 3.20569192]]
[[ 4.78251811]]
[[ 4.32022545]]
[[ 4.21915839]]
[[ 4.11136019]]
[[ 4.44525889]]
[[ 4.12931924]]
[[ 4.23784839]]
[[ 3.83748121]]
[[ 4.77292692]]
[[ 4.50273219]]
[[ 3.89119167]]
[[ 4.32686354]]
[[ 4.81139621]]
[[ 4.29381496]]
[[ 4.49611355]]
[[ 3.61156633]]
[[ 4.0877287]]
[[ 3.7310213]]
[[ 4.08334716]]
[[ 4.94768382]]
[[ 4.12961803]]
[[ 4.43688858]]
[[ 3.29103435]]
[[ 4.13200271]]
[[ 4.17170438]]
[[ 4.34634597]]
[[ 3.92865053]]
[[ 3.80391731]]
[[ 3.68999368]]
[[ 3.84905171]]
[[ 4.34805878]]
[[ 4.78693526]]
[[ 4.1289063]]
[[ 3.56400297]]
[[ 

[[ 4.53212194]]
[[ 3.71698539]]
[[ 4.06465362]]
[[ 4.52536108]]
[[ 3.58106501]]
[[ 4.11442]]
[[ 4.89041719]]
[[ 4.17283731]]
[[ 4.03650586]]
[[ 4.11418328]]
[[ 3.61116294]]
[[ 4.00013039]]
[[ 4.63617616]]
[[ 4.74081688]]
[[ 4.13934416]]
[[ 3.78495111]]
[[ 4.38106159]]
[[ 4.48907448]]
[[ 3.24054118]]
[[ 3.8976429]]
[[ 4.44786055]]
[[ 4.0827736]]
[[ 3.82264014]]
[[ 4.22744212]]
[[ 3.97150338]]
[[ 5.07711569]]
[[ 4.27264444]]
[[ 4.40429948]]
[[ 3.80994001]]
[[ 4.70744105]]
[[ 4.45331536]]
[[ 4.4825723]]
[[ 3.97990878]]
[[ 4.53427085]]
[[ 4.91033793]]
[[ 4.31953952]]
[[ 3.06369522]]
[[ 4.0983312]]
[[ 4.14157783]]
[[ 4.50481979]]
[[ 4.9288934]]
[[ 3.86870201]]
[[ 4.81794127]]
[[ 3.7757379]]
[[ 3.9728709]]
[[ 4.83419302]]
[[ 3.29762039]]
[[ 3.53902344]]
[[ 3.9232306]]
[[ 4.27196149]]
[[ 3.86173887]]
[[ 4.65052789]]
[[ 4.64199597]]
[[ 4.04338234]]
[[ 4.50256535]]
[[ 4.08691009]]
[[ 4.22805061]]
[[ 4.30326547]]
[[ 4.50954899]]
[[ 4.16536378]]
[[ 4.1741603]]
[[ 4.53637604]]
[[ 3.79216456]]
[[ 3

[[ 4.28379773]]
[[ 3.97661556]]
[[ 4.18573048]]
[[ 4.09673314]]
[[ 4.06278705]]
[[ 4.31288001]]
[[ 4.56193138]]
[[ 4.09792278]]
[[ 4.70486963]]
[[ 4.37343236]]
[[ 4.73582493]]
[[ 4.1757278]]
[[ 3.80254211]]
[[ 3.88056139]]
[[ 4.08786973]]
[[ 4.87215052]]
[[ 4.60172553]]
[[ 4.32624302]]
[[ 4.12583457]]
[[ 4.84341888]]
[[ 3.87921772]]
[[ 4.8099492]]
[[ 4.61456578]]
[[ 3.83663153]]
[[ 4.48480623]]
[[ 4.12338659]]
[[ 3.31270133]]
[[ 3.54797183]]
[[ 3.83102474]]
[[ 3.95900975]]
[[ 4.21462802]]
[[ 3.38184805]]
[[ 4.17568076]]
[[ 4.20695463]]
[[ 3.7573975]]
[[ 4.81974005]]
[[ 4.50297139]]
[[ 4.22603695]]
[[ 3.91961396]]
[[ 4.26115572]]
[[ 3.83176086]]
[[ 3.71656121]]
[[ 4.53072547]]
[[ 3.54083527]]
[[ 4.35095429]]
[[ 3.9048878]]
[[ 4.05472067]]
[[ 4.5100872]]
[[ 4.7123587]]
[[ 2.8863483]]
[[ 3.46852463]]
[[ 4.09076259]]
[[ 4.61306856]]
[[ 3.5891654]]
[[ 4.76303933]]
[[ 4.25139084]]
[[ 5.07716452]]
[[ 4.49494396]]
[[ 3.32140977]]
[[ 4.69253196]]
[[ 3.80866511]]
[[ 4.78331917]]
[[ 4.14454032]]


[[ 4.45925113]]
[[ 4.32000624]]
[[ 4.33490903]]
[[ 4.33553978]]
[[ 4.54974773]]
[[ 4.07100152]]
[[ 3.64105163]]
[[ 4.39371999]]
[[ 4.47755229]]
[[ 4.55680035]]
[[ 4.25637196]]
[[ 4.04409516]]
[[ 4.33812831]]
[[ 4.48419518]]
[[ 4.20398793]]
[[ 4.89448031]]
[[ 3.68855426]]
[[ 4.26736149]]
[[ 4.89690249]]
[[ 3.86519926]]
[[ 4.14039367]]
[[ 3.78154328]]
[[ 4.36653214]]
[[ 4.15311216]]
[[ 3.38278265]]
[[ 4.05564236]]
[[ 4.64800847]]
[[ 4.24540848]]
[[ 4.37401117]]
[[ 4.40263401]]
[[ 4.05649116]]
[[ 4.05439334]]
[[ 3.85842411]]
[[ 3.63432051]]
[[ 4.71789267]]
[[ 4.53677964]]
[[ 3.97919841]]
[[ 4.71638455]]
[[ 4.82721057]]
[[ 3.32775673]]
[[ 4.28308733]]
[[ 4.40288474]]
[[ 3.86029137]]
[[ 4.72666933]]
[[ 4.10155844]]
[[ 4.49528542]]
[[ 4.32871662]]
[[ 4.40466453]]
[[ 4.28715798]]
[[ 4.48242049]]
[[ 3.749014]]
[[ 4.12204675]]
[[ 4.0315188]]
[[ 4.61607675]]
[[ 4.61847519]]
[[ 4.23281203]]
[[ 4.01967494]]
[[ 4.6304826]]
[[ 4.20015991]]
[[ 4.44171605]]
[[ 3.87920068]]
[[ 3.68153598]]
[[ 4.3564877

[[ 4.5935168]]
[[ 3.69816448]]
[[ 4.34493797]]
[[ 4.50412795]]
[[ 4.66892933]]
[[ 2.56603832]]
[[ 4.54337702]]
[[ 4.02408308]]
[[ 3.9758771]]
[[ 3.87653542]]
[[ 4.63381356]]
[[ 4.44738047]]
[[ 4.26272432]]
[[ 3.75378489]]
[[ 4.8370052]]
[[ 3.59976438]]
[[ 4.06411519]]
[[ 3.67870297]]
[[ 4.30432898]]
[[ 4.22041099]]
[[ 4.56356331]]
[[ 4.34753807]]
[[ 4.12221051]]
[[ 3.97430516]]
[[ 3.7727373]]
[[ 3.69542757]]
[[ 4.55913769]]
[[ 3.97175516]]
[[ 4.49303873]]
[[ 3.27130197]]
[[ 4.07478309]]
[[ 3.89005232]]
[[ 3.67932726]]
[[ 4.30598138]]
[[ 4.2783272]]
[[ 4.39216158]]
[[ 4.26168835]]
[[ 4.43472394]]
[[ 4.47781691]]
[[ 4.73008058]]
[[ 4.15430039]]
[[ 3.90010876]]
[[ 4.68558941]]
[[ 4.20139562]]
[[ 4.70583771]]
[[ 4.36984177]]
[[ 4.16726482]]
[[ 3.33247428]]
[[ 3.46047491]]
[[ 3.29800315]]
[[ 3.9099318]]
[[ 4.25822683]]
[[ 4.17962334]]
[[ 4.75115445]]
[[ 4.00195717]]
[[ 4.04151011]]
[[ 4.43921626]]
[[ 4.47867469]]
[[ 3.81611457]]
[[ 4.24275807]]
[[ 3.95032166]]
[[ 4.03398434]]
[[ 3.65776188]

...

[[ 4.02954684]]
[[ 4.37413015]]
[[ 4.60269464]]
[[ 4.01438489]]
[[ 3.72088509]]
[[ 4.4261452]]
[[ 4.49220439]]
[[ 4.49171099]]
[[ 3.78240106]]
[[ 4.73611515]]
[[ 3.90914195]]
[[ 4.21166532]]
[[ 3.64920401]]
[[ 4.64196163]]
[[ 4.08859625]]
[[ 4.52676246]]
[[ 3.83906067]]
[[ 4.63202144]]
[[ 3.31180278]]
[[ 3.99769699]]
[[ 3.99376087]]
[[ 4.18439502]]
[[ 4.1662818]]
[[ 4.47582573]]
[[ 3.60686254]]
[[ 3.26637368]]
[[ 4.33074115]]
[[ 3.75420077]]
[[ 3.48954351]]
[[ 3.30211623]]
[[ 4.42499564]]
[[ 4.57076273]]
[[ 3.47602694]]
[[ 4.61452364]]
[[ 4.32145854]]
[[ 3.91963612]]
[[ 3.91057917]]
[[ 4.43888519]]
[[ 3.95208815]]
[[ 3.86679363]]
[[ 3.59226372]]
[[ 4.31165047]]
[[ 4.42018519]]
[[ 4.76307423]]
[[ 3.85443499]]
[[ 4.91649671]]
[[ 4.13448986]]
[[ 3.83532733]]
[[ 4.26052155]]
[[ 3.86487027]]
[[ 4.17376088]]
[[ 4.63733985]]
[[ 3.80999757]]
[[ 4.11351631]]
[[ 4.24802008]]
[[ 4.68224315]]
[[ 4.53324422]]
[[ 3.77478416]]
[[ 4.59700446]]
[[ 4.75747174]]
[[ 4.72426086]]
[[ 4.09024399]]
[[ 4.47994

[[ 3.64292368]]
[[ 5.15639664]]
[[ 3.74103618]]
[[ 4.10988091]]
[[ 4.03315795]]
[[ 3.79681005]]
[[ 3.96884061]]
[[ 3.97701975]]
[[ 4.26446777]]
[[ 4.49767516]]
[[ 3.76406644]]
[[ 4.42892611]]
[[ 4.13797001]]
[[ 4.68721069]]
[[ 3.94914616]]
[[ 3.74228616]]
[[ 4.37598854]]
[[ 4.73781087]]
[[ 4.42755425]]
[[ 3.94741528]]
[[ 4.86004907]]
[[ 3.97642383]]
[[ 3.83525841]]
[[ 4.63415416]]
[[ 4.24770425]]
[[ 4.90625326]]
[[ 4.74880658]]
[[ 4.18829733]]
[[ 4.11081619]]
[[ 4.30378354]]
[[ 4.33555853]]
[[ 3.97475796]]
[[ 4.23010035]]
[[ 4.24909262]]
[[ 4.07031834]]
[[ 4.4713708]]
[[ 4.55929452]]
[[ 4.41209606]]
[[ 4.08053216]]
[[ 4.36539653]]
[[ 4.18245834]]
[[ 4.20163227]]
[[ 3.95390267]]
[[ 4.34021205]]
[[ 4.73022355]]
[[ 4.3118049]]
[[ 4.71113565]]
[[ 4.68343829]]
[[ 4.59272407]]
[[ 4.0394349]]
[[ 3.9424837]]
[[ 3.73848972]]
[[ 4.0140979]]
[[ 4.48930342]]
[[ 4.16316144]]
[[ 4.52956293]]
[[ 4.06896477]]
[[ 4.21981648]]
[[ 4.65447019]]
[[ 4.46990683]]
[[ 4.06335022]]
[[ 4.04464823]]
[[ 4.60103624

[[ 4.73161819]]
[[ 4.56461674]]
[[ 3.85965914]]
[[ 4.34662651]]
[[ 4.17103849]]
[[ 4.54944222]]
[[ 4.90757086]]
[[ 4.28821825]]
[[ 4.15190129]]
[[ 4.83539967]]
[[ 4.69887564]]
[[ 4.61372089]]
[[ 4.28869476]]
[[ 4.53430896]]
[[ 3.88300204]]
[[ 3.64072621]]
[[ 4.85559318]]
[[ 3.43205848]]
[[ 4.46006804]]
[[ 3.73079353]]
[[ 3.77966534]]
[[ 4.91511186]]
[[ 4.12144485]]
[[ 4.49593059]]
[[ 4.16082205]]
[[ 4.83225651]]
[[ 4.36380924]]
[[ 4.45990106]]
[[ 4.0015779]]
[[ 4.61950657]]
[[ 4.00821747]]
[[ 3.94438149]]
[[ 3.7784051]]
[[ 4.31163782]]
[[ 4.42589619]]
[[ 4.36565051]]
[[ 4.01461318]]
[[ 4.08109908]]
[[ 4.72857738]]
[[ 4.20406585]]
[[ 3.01045102]]
[[ 4.53757229]]
[[ 3.88072313]]
[[ 3.92679935]]
[[ 4.04931882]]
[[ 3.81239558]]
[[ 3.27043995]]
[[ 4.1971972]]
[[ 4.0003923]]
[[ 3.52271676]]
[[ 3.85327448]]
[[ 4.25413633]]
[[ 4.34173899]]
[[ 4.37339333]]
[[ 4.22158698]]
[[ 4.02029593]]
[[ 4.59126319]]
[[ 3.95866695]]
[[ 3.81231323]]
[[ 4.27609172]]
[[ 3.83343447]]
[[ 3.84225773]]
[[ 4.0583898

[[ 4.41182846]]
[[ 3.58098185]]
[[ 4.27025111]]
[[ 4.2459386]]
[[ 4.60890647]]
[[ 3.89305287]]
[[ 4.79577965]]
[[ 4.09379638]]
[[ 4.18528792]]
[[ 3.96920535]]
[[ 4.06530783]]
[[ 4.62733587]]
[[ 4.05190649]]
[[ 3.75770572]]
[[ 4.04595232]]
[[ 4.13385749]]
[[ 4.34959287]]
[[ 4.58208967]]
[[ 3.74487051]]
[[ 4.77909875]]
[[ 4.34899217]]
[[ 4.56834579]]
[[ 4.69312472]]
[[ 4.08305151]]
[[ 4.45445998]]
[[ 4.41558542]]
[[ 4.06546681]]
[[ 4.07648925]]
[[ 4.61801807]]
[[ 4.65754995]]
[[ 4.77382005]]
[[ 5.20732896]]
[[ 3.41054773]]
[[ 3.95137729]]
[[ 3.97932339]]
[[ 4.43696178]]
[[ 4.45721874]]
[[ 3.37123897]]
[[ 4.68480276]]
[[ 4.5299742]]
[[ 4.33839324]]
[[ 4.61104649]]
[[ 4.12879958]]
[[ 4.17203287]]
[[ 3.80953289]]
[[ 4.19509252]]
[[ 4.31555847]]
[[ 3.31959467]]
[[ 4.42109609]]
[[ 4.72056964]]
[[ 4.04091018]]
[[ 3.95989821]]
[[ 4.35071754]]
[[ 3.64343402]]
[[ 4.22938051]]
[[ 4.94389034]]
[[ 3.23647908]]
[[ 3.96465606]]
[[ 4.10791404]]
[[ 4.18505326]]
[[ 3.81892028]]
[[ 3.53662251]]
[[ 4.47014

[[ 4.41674277]]
[[ 4.58704]]
[[ 3.8232193]]
[[ 3.6523368]]
[[ 3.65960512]]
[[ 4.43526324]]
[[ 4.57110626]]
[[ 3.85328349]]
[[ 4.23987716]]
[[ 3.97328828]]
[[ 4.33835874]]
[[ 3.8578583]]
[[ 4.08941699]]
[[ 4.06739166]]
[[ 3.72822515]]
[[ 3.9292755]]
[[ 4.82152126]]
[[ 4.87326591]]
[[ 4.62732626]]
[[ 4.34736104]]
[[ 4.74771322]]
[[ 4.37725442]]
[[ 4.07319579]]
[[ 4.02928552]]
[[ 4.47106512]]
[[ 4.2685711]]
[[ 4.45383794]]
[[ 4.28708296]]
[[ 3.38404229]]
[[ 4.5355189]]
[[ 4.25890806]]
[[ 4.74019979]]
[[ 3.60278263]]
[[ 4.15960385]]
[[ 4.4430376]]
[[ 4.41366636]]
[[ 4.83963956]]
[[ 3.42990728]]
[[ 4.89206428]]
[[ 4.02067389]]
[[ 3.78382495]]
[[ 4.71982172]]
[[ 4.06746187]]
[[ 4.22043565]]
[[ 4.32852683]]
[[ 4.67368013]]
[[ 4.5512142]]
[[ 3.77074845]]
[[ 3.85375382]]
[[ 4.38347205]]
[[ 4.40390121]]
[[ 3.57241427]]
[[ 4.49699055]]
[[ 4.6374809]]
[[ 4.51284024]]
[[ 4.47178565]]
[[ 3.50755666]]
[[ 3.6875747]]
[[ 4.25943919]]
[[ 4.09123092]]
[[ 3.58725304]]
[[ 4.54396256]]
[[ 3.83059712]]
[[ 4.

In [ ]:
predictValid()

In [26]:
np.matmul(gammaU['U455762636'],np.transpose(gammaI['B408037852']))[0][0] + betaI['B408037852']

array([[ 0.05303588]])